In [5]:
import numpy as np
import pandas as pd
import polars as pl
import glob

In [6]:
# read all data
players = pl.read_csv('../nfl-big-data-bowl-2024/players.csv')
plays = pl.read_csv('../nfl-big-data-bowl-2024/plays.csv',infer_schema_length=100000)
games = pl.read_csv('../nfl-big-data-bowl-2024/games.csv',infer_schema_length=10000)
tracking = pl.read_csv('../nfl-big-data-bowl-2024/tracking_week*.csv',infer_schema_length=10000)

# transformations
replacement_values = {'NA': '0'}
tracking = tracking.with_columns(
    pl.col('o').apply(lambda x: replacement_values.get(x, x)),
    pl.col('dir').apply(lambda x: replacement_values.get(x, x)),
)

tracking=tracking.with_columns([
    pl.when(pl.col('playDirection')=='right').then(pl.col('o').cast(pl.Float64)).otherwise((180+pl.col('o').cast(pl.Float64))%360).alias('firstAdjustedO'),
])

tracking=tracking.with_columns([
    pl.when(pl.col('firstAdjustedO') <= 180).then(180-pl.col('firstAdjustedO')).otherwise(540-pl.col('firstAdjustedO')).alias('adjustedO')
])

In [7]:
# Setup for vis

from PIL import Image, ImageDraw, ImageFont
import numpy as np
import copy

colors = {
    'ARI':"#97233F", 
    'ATL':"#A71930", 
    'BAL':'#241773', 
    'BUF':"#00338D", 
    'CAR':"#0085CA", 
    'CHI':"#C83803", 
    'CIN':"#FB4F14", 
    'CLE':"#311D00", 
    'DAL':'#003594',
    'DEN':"#FB4F14", 
    'DET':"#0076B6", 
    'GB':"#203731", 
    'HOU':"#03202F", 
    'IND':"#002C5F", 
    'JAX':"#9F792C", 
    'KC':"#E31837", 
    'LA':"#003594", 
    'LAC':"#007FC8", 
    'LV':"#000000",
    'MIA':"#008E97", 
    'MIN':"#4F2683", 
    'NE':"#002244", 
    'NO':"#D3BC8D", 
    'NYG':"#0B2265", 
    'NYJ':"#125740", 
    'PHI':"#004C54", 
    'PIT':"#FFB612", 
    'SEA':"#69BE28", 
    'SF':"#AA0000",
    'TB':'#D50A0A', 
    'TEN':"#4B92DB", 
    'WAS':"#5A1414", 
    'football':'#CBB67C'
}



In [37]:
# Drawing functions to create the field
def get_blank_field():
    yardlines = np.arange(100, 1100+1, 100)
    yardline_width = 4
    
    yard_mark = list(np.arange(0, 50, 10)) + [50] + list(reversed(list(np.arange(0, 50, 10))))
    font_size=40

    # Draw a green rectangle
    field = Image.new("RGB", (1200, 533), "green")
    draw = ImageDraw.Draw(field)
    
    # Draw the yardlines and the yard marker text
    assert yardline_width % 2 == 0
    for yl, ym in zip(yardlines, yard_mark):
        yl_x = (yl - (yardline_width / 2))
        draw.line([(yl_x, 0), (yl_x, 533)], width = yardline_width, fill="white")
        
        font = ImageFont.load_default()
        draw.text((yl-(font_size/2), 533-(font_size+5)), str(ym), font=font, fill = "black")
    
    # Flip the image so the text is right side up
    field = field.transpose(1)

    return field

def draw_play_frame(frame, highlight_ids = []):

    field = get_blank_field()
    draw = ImageDraw.Draw(field)

    p_rad = 4
    padding=2
    fb_w=8
    fb_h=5
    
    df = copy.deepcopy(frame)
    
    # Round the player locations to work in the image coordinates
    plot_x = df.loc[:, "x"].apply(lambda x: round(x, 1) * 10)
    df.loc[:, "plot_x"] = plot_x
    plot_y = df.loc[:, "y"].apply(lambda x: round(x, 1) * 10)
    df.loc[:, "plot_y"] = plot_y
        
    for row in df.iterrows():
        
        x = row[1]["plot_x"]
        y = row[1]["plot_y"]
        
        # Draw a white circle behind any player dots to be highlighted
        if row[1]["nflId"] in highlight_ids:
            draw.ellipse(((x-p_rad)-padding, (y-p_rad)-padding, (x+p_rad)+padding, (y+p_rad)+padding), fill="white")

        # Draw the football
        if row[1]["club"] == "football":
            draw.ellipse((x-fb_w, y-fb_h, x+fb_w, y+fb_h), fill=colors[row[1]["club"]])
        # Draw the players with color according to the colors dictionary
        else:
            if row[1]["jerseyNumber"] in highlight_ids:    
                vision_cone_coordinates = get_vision_cone_coordinates(row[1])
                draw.line([vision_cone_coordinates[0], vision_cone_coordinates[1]])
                draw.line([vision_cone_coordinates[0], vision_cone_coordinates[2]])

                print(vision_cone_coordinates)
                
                draw.arc([vision_cone_coordinates[2], vision_cone_coordinates[1]], 0, 60, width=2)
                
                #draw.polygon(vision_cone_coordinates, outline='black')
                
            draw.ellipse((x-p_rad, y-p_rad, x+p_rad, y+p_rad), fill=colors[row[1]["club"]])
        
    return field

def finalize(field, min_x = None, max_x = None):
    """
    Finalizes the image. Does the following
    - Flips the image along the x axis
    - Optionally crops out empty field according to min_x, max_x
    """
    field = field.transpose(1)
    if (min_x is not None) & (max_x is not None):
        field = field.crop((min_x, 0, max_x, 533))

    return field

def create_play_gif(play_player_tracking_df, gif_name, crop=False, highlight_ids=[]):
    """
    Draws the play frame by frame and saves to gif
    
    Parameters
    play_player_tracking_df - A df of player_tracking_data that contains 
    a unique gameId and a unique playId
    gif_name - The name of the gif, minus the .gif extension. This is 
    added automatically.
    crop - Whether or not to crop the gif to only contain the minimum and
    maximum x values within the entire play
    highlight_ids - The ids of players to draw a white circle behind them in
    order to call attention to them.
    
    """
    min_x = (round(play_player_tracking_df.x.min(), 1) * 10) - 50
    max_x = (round(play_player_tracking_df.x.max(), 1) * 10) + 50
    
    gif_frames = []
    frames = play_player_tracking_df["frameId"].values
    for i in frames:
        frame = play_player_tracking_df[play_player_tracking_df["frameId"] == i].copy(deep=True)
        field = get_blank_field()
        
        field = draw_play_frame(frame, highlight_ids)
            
        if crop:
            field = finalize(field, min_x = min_x, max_x =  max_x)
        else:
            field = finalize(field)
            
        gif_frames.append(field)
    frame_one = gif_frames[0]
    frame_one.save(f"{gif_name}.gif", format="GIF", append_images=gif_frames,
                save_all=True, duration=100, loop=0)

# Note: dist/angle aren't exactly what we use in code and is overexaggerated for show
# for some reason 120 looks a lot nicer than 60 can math this out and figure it out time permitting
def get_vision_cone_coordinates(player):
    import math
    DIST = 10
    ANGLE = 120

    player_x = player['plot_x']
    player_y = player['plot_y']
    player_orientation = float(player['adjustedO'])
    
    vision_cone_coordinates = []
    vision_cone_coordinates.append((player_x, player_y))

    high_on_potenuse = abs(DIST / math.cos(math.radians(ANGLE)))
    
    x1 = player_x + high_on_potenuse * math.cos(math.radians(player_orientation))
    y1 = player_y + high_on_potenuse * math.sin(math.radians(player_orientation))

    x2 = player_x + high_on_potenuse * math.cos(math.radians(player_orientation-ANGLE))
    y2 = player_y + high_on_potenuse * math.sin(math.radians(player_orientation-ANGLE))

    # print("frame number: ", player['frameId'])
    # print("hyp1: ", high_on_potenuse, " player_orientation: ", player_orientation, " original o: ", player['o'])
    # print("x1: ", x1, " y1: ", y1)
    # print("x2: ", x2, " y2: ", y2)

    vision_cone_coordinates.append((x1, y1))
    vision_cone_coordinates.append((x2, y2))

    return vision_cone_coordinates

    
    

In [38]:
gif_df = tracking.filter(pl.col("gameId")==2022091107).filter(pl.col("playId")==959).filter(pl.col('frameId')==1)
create_play_gif(gif_df.to_pandas(), "./test", crop=False, highlight_ids=["42", "52"])

[(359.0, 363.0), (363.8993794135044, 382.39061838525345), (373.3430784099669, 349.0617037725)]


ValueError: x1 must be greater than or equal to x0